In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import plotly.graph_objs as go

from binance.client import Client

In [2]:
# loading binance parameters from environment variables
load_dotenv()
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

# setting up Binance client
client = Client(api_key=api_key, api_secret=api_secret)

In [3]:
# getting binance data - refer to binance package documentation for more info
data = client.get_historical_klines('SUSHIUSDT', interval=client.KLINE_INTERVAL_1HOUR, start_str='20d')

In [4]:
# putting data into a table
headers = ['date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']
df = pd.DataFrame(data=data, columns=headers)

df.head()

,date,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,1642212000000,7.08100000,7.09900000,7.04400000,7.09500000,95979.80000000,1642215599999,678941.19980000,1851,35916.40000000,254063.28670000,0
1,1642215600000,7.09500000,7.16400000,7.09300000,7.13800000,160034.40000000,1642219199999,1140954.06300000,3621,68763.50000000,490152.99510000,0
2,1642219200000,7.13200000,7.13300000,7.06000000,7.10400000,83550.30000000,1642222799999,592543.67020000,1712,38132.40000000,270372.84510000,0
3,1642222800000,7.10400000,7.11000000,7.04900000,7.06400000,46300.30000000,1642226399999,327402.52900000,1152,21858.40000000,154556.95470000,0
4,1642226400000,7.06600000,7.12700000,7.00600000,7.11600000,112146.80000000,1642229999999,792859.72920000,2593,54969.40000000,388730.42020000,0


In [5]:
# converting timestamps into datetime for better visualization
df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:$S'))
df['close_time'] = df['close_time'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:$S'))

In [6]:
# converting prices and volumes into floats
df[['open', 'high', 'low', 'close', 'volume', 'taker_base_vol', 'taker_quote_vol']] = df[['open', 'high', 'low', 'close', 'volume', 'taker_base_vol', 'taker_quote_vol']].astype('float64')

In [7]:
#using the newly updated date as our index
df.set_index('date', inplace=True)

df.head()

,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
date,,,,,,,,,,,
2022-01-14 21:00:$S,7.081,7.099,7.044,7.095,95979.8,2022-01-14 21:59:$S,678941.19980000,1851,35916.4,254063.2867,0
2022-01-14 22:00:$S,7.095,7.164,7.093,7.138,160034.4,2022-01-14 22:59:$S,1140954.06300000,3621,68763.5,490152.9951,0
2022-01-14 23:00:$S,7.132,7.133,7.060,7.104,83550.3,2022-01-14 23:59:$S,592543.67020000,1712,38132.4,270372.8451,0
2022-01-15 00:00:$S,7.104,7.110,7.049,7.064,46300.3,2022-01-15 00:59:$S,327402.52900000,1152,21858.4,154556.9547,0
2022-01-15 01:00:$S,7.066,7.127,7.006,7.116,112146.8,2022-01-15 01:59:$S,792859.72920000,2593,54969.4,388730.4202,0


In [8]:
# creating EWMA data points using pandas methods ewm and rolling
df['MA5'] = df['close'].ewm(span=12, adjust=False).mean()
df['MA20'] = df['close'].rolling(24).mean()

df.tail()

,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,MA5,MA20
date,,,,,,,,,,,,,
2022-02-03 16:00:$S,3.952,4.050,3.952,4.032,225216.1,2022-02-03 16:59:$S,903841.51100000,2677,101230.2,405654.4570,0,4.017237,4.038042
2022-02-03 17:00:$S,4.034,4.046,4.014,4.031,103346.5,2022-02-03 17:59:$S,416388.94610000,1088,43740.1,176205.5077,0,4.019354,4.035417
2022-02-03 18:00:$S,4.032,4.099,4.020,4.096,161937.3,2022-02-03 18:59:$S,656838.40050000,1892,97219.7,394834.5169,0,4.031146,4.035917
2022-02-03 19:00:$S,4.096,4.100,4.055,4.064,92153.5,2022-02-03 19:59:$S,375583.08110000,1154,48127.1,196189.8680,0,4.036200,4.034625
2022-02-03 20:00:$S,4.063,4.065,4.055,4.060,3326.1,2022-02-03 20:59:$S,13498.99500000,60,1889.8,7670.0778,0,4.039862,4.034708


In [9]:
# plot this bad boy
fig = go.Figure()

fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='market data'))
fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], line={'color': 'blue','width': 1.5}, name='Long Term MA'))
fig.add_trace(go.Scatter(x=df.index, y=df['MA5'], line={'color': 'orange','width': 1.5}, name='Short Term MA'))

fig.show()